In [1]:
import pandas as pd

all_tweets_df = pd.read_csv('all_tweets')

In [2]:
from sklearn.model_selection import train_test_split

# Training = 0.7, validation = 0.1, test = 0.2
training_tweets, temp_tweets, training_labels, temp_labels = train_test_split(list(all_tweets_df['text']),list(all_tweets_df['condition']), test_size=0.3, random_state=123)
val_tweets, test_tweets, val_labels, test_labels = train_test_split(temp_tweets, temp_labels, test_size=(1/3), random_state=123)

In [3]:
from transformers import RobertaTokenizer, RobertaModel

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

def tokenize_function(dataset):
    model_inputs = tokenizer(dataset[0], padding="max_length", truncation=True)
    return model_inputs


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
# train_dataset = [{'text': tweet} for tweet in training_tweets],\
#                 [{'labels': label} for label in training_labels]
train_dataset = list(zip(training_tweets[:10], training_labels[:10]))
# val_dataset = list(zip(val_tweets,val_labels))
# test_dataset = list(zip(test_tweets,test_labels))

In [23]:
x=map(tokenize_function, train_dataset)

In [ ]:
train_dataset[,:]

In [12]:
train_dataset_df = train_dataset.map(tokenize_function, batched=True)
# val_dataset = val_dataset.map(tokenize_function, batched=True)
# test_dataset = test_dataset.map(tokenize_function, batched=True)

AttributeError: 'DataFrame' object has no attribute 'map'